In [69]:
MARKET = 'NSE'

In [70]:
import sys
import pathlib
import numpy as np
import pandas as pd
import yaml
import asyncio

from ib_insync import IB, util, Option, MarketOrder, Contract
from typing import Callable, Coroutine, Union

In [71]:
# Specific to Jupyter. Will be ignored in IDE / command-lines
import IPython as ipy
if ipy.get_ipython().__class__.__name__ == 'ZMQInteractiveShell':
    import nest_asyncio
    nest_asyncio.apply()
    util.startLoop()
    pd.options.display.max_columns = None
    pd.options.display.float_format = '{:,.2f}'.format # set float precision with comma
    
    THIS_FOLDER = '' # Dummy for jupyter notebook's current folder
    BAR_FORMAT = "{l_bar}{bar:-20}{r_bar}"

In [72]:
# Get capability to import programs from `asyncib` folder
cwd = pathlib.Path.cwd() # working directory from where python was initiated
DATAPATH = cwd.joinpath('data', MARKET.lower()) # path to store data files
LOGFILE = cwd.joinpath(THIS_FOLDER, 'data', 'log', 'temp.log') # path to store log files

IBPATH = cwd.parent.parent.joinpath('asyncib') # where ib programs are stored

# append IBPATH to import programs.
if str(IBPATH) not in sys.path:  # Convert it to string!
    sys.path.append(str(IBPATH))
    
IBDATAPATH = IBPATH.joinpath('data', MARKET.lower())

In [73]:
# Get the host, port, cid
from engine import Vars
from ib_insync import Option

ibp = Vars(MARKET.upper())  # IB Parameters from var.yml
HOST, PORT, CID = ibp.HOST, ibp.PORT, ibp.CID

In [74]:
# Get the pickle files
from os import listdir
fs = listdir(DATAPATH)

files = [f for f in fs if f[-4:] == '.pkl']
for f in files:
    exec(f"{f.split('.')[0]} = pd.read_pickle(DATAPATH.joinpath(f))")
np.sort(np.array(files))

array(['df_chains.pkl', 'df_nakeds.pkl', 'df_ohlcs.pkl',
       'df_opt_prices.pkl', 'df_opts.pkl', 'df_symlots.pkl',
       'df_unds.pkl', 'dfrq.pkl', 'qopt_rejects.pkl', 'qopts.pkl'],
      dtype='<U17')

In [83]:
# * IMPORTS
from engine import get_unds, get_chains, executeAsync, qualify, post_df, price, get_margins, get_ohlcs
from support import Vars, quick_price

In [84]:
# * FUNCTION INPUTS
MARKET = 'NSE'
SYMBOL = 'NIFTY50'
TARGET_DTE = 20
RUN_ON_PAPER = True
SAVE = True

In [85]:
# * SETUP
# ... set parameters from var.yml
ibp = Vars(MARKET.upper())

HOST, CID = ibp.HOST, ibp.CID
if RUN_ON_PAPER:
    PORT = ibp.PAPER
else:
    PORT = ibp.PORT
    
DATAPATH = pathlib.Path.cwd().joinpath(THIS_FOLDER, "data", MARKET.lower())
LOGPATH = pathlib.Path.cwd().joinpath(THIS_FOLDER, "data", "log")

# * SETUP LOGS AND CLEAR THEM
LOGFILE = LOGPATH.joinpath(MARKET.lower() + "_dive.log")
util.logToFile(path=LOGFILE, level=30)
with open(LOGFILE, "w"):
    pass

In [86]:
# * GET THE UNDERLYING
df_symlots = pd.read_pickle(DATAPATH.joinpath("df_symlots.pkl"))
df_symlots = df_symlots[df_symlots.symbol == SYMBOL]

In [87]:
%%time
# . Get underlying price
und_ct = df_symlots.contract.iloc[0]
with IB().connect(HOST, PORT, CID) as ib:
    df_und_pr = ib.run(quick_price(ib, und_ct))

undPrice = df_und_pr.price.iloc[0]

# * MAKE THE OPTIONS NEAREST TO TARGET_DTE
df_ch = df_chains[df_chains.symbol == SYMBOL].drop('mult', 1)

# . get strikes for nearest DTE
nearest_dte = df_ch.dte.transform(lambda x: abs(x-TARGET_DTE)).min()
df_ch1 = df_ch[abs(df_ch.dte - TARGET_DTE) == abs(df_ch.dte - TARGET_DTE).min()].reset_index(drop=True)

# . integrate und_price
df_ch1['undPrice'] = undPrice

# . get the right
df_ch1['right'] = np.where(df_ch1.undPrice > df_ch1.strike, 'C', 'P')

# . make the raw options
raw_opts = [
    Option(s, e, k, r, x)
    for s, e, k, r, x in zip(
        df_ch1.symbol,
        df_ch1.expiry,
        df_ch1.strike,
        df_ch1.right,
        ["NSE" if MARKET.upper() == "NSE" else "SMART"] * len(df_ch1),
    )
]

# . qualify raw options
with IB().connect(HOST, PORT, CID) as ib:
    qopts = ib.run(executeAsync(
                    ib=ib,
                    algo=qualify,
                    cts=raw_opts,
                    CONCURRENT=200,
                    TIMEOUT=5,
                    post_process=post_df,
                    SHOW_TQDM=False))

# . get prices
with IB().connect(HOST, PORT, CID) as ib:
    df_opt_prices = ib.run(
        executeAsync(
            ib=ib,
            algo=price,
            cts=qopts,
            post_process=post_df,
            CONCURRENT=40 * 4,
            TIMEOUT=11,
            DATAPATH=DATAPATH,
            REUSE=False,
            OP_FILENAME="",
            **{"FILL_DELAY": 11},
        )
    )


No bid-ask for NIFTY50 of secType: IND



Wall time: 16 s


In [93]:
df_opt_prices

,secType,conId,symbol,expiry,strike,right,localSymbol,contract,time,greeks,bid,ask,close,last,price,iv
0,OPT,459327621,NIFTY50,20210107,"14,850.00",P,NIFTY2110714850PE,"Option(conId=459327621, symbol='NIFTY50', last...",2020-12-18 09:24:00.251031+00:00,"(0.16841493341581615, -0.9696159040969264, 107...",899.10,"1,221.50","1,115.15",nan,"1,115.15",0.17
1,OPT,459089067,NIFTY50,20210107,"14,700.00",P,NIFTY2110714700PE,"Option(conId=459089067, symbol='NIFTY50', last...",2020-12-18 09:24:00.252029+00:00,"(0.1582034836255517, -0.9577100429559546, 926....",808.90,"1,072.40",982.65,nan,982.65,0.16
2,OPT,453924350,NIFTY50,20210107,"12,850.00",C,NIFTY2110712850CE,"Option(conId=453924350, symbol='NIFTY50', last...",2020-12-18 09:24:00.256017+00:00,"(0.2134774861582349, 0.9226385757033658, 953.1...",817.60,"1,083.90",969.85,nan,969.85,0.21
3,OPT,454836927,NIFTY50,20210107,"14,250.00",P,NIFTY2110714250PE,"Option(conId=454836927, symbol='NIFTY50', last...",2020-12-18 09:24:00.251031+00:00,"(0.13938013060758744, -0.8437554738417199, 504...",365.35,597.20,622.35,nan,622.35,0.14
4,OPT,457243522,NIFTY50,20210107,"14,500.00",P,NIFTY2110714500PE,"Option(conId=457243522, symbol='NIFTY50', last...",2020-12-18 09:24:00.251031+00:00,"(0.14793150592771773, -0.9266947393987865, 733...",627.65,822.60,814.65,nan,814.65,0.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,OPT,453923990,NIFTY50,20210107,"10,600.00",C,NIFTY2110710600CE,"Option(conId=453923990, symbol='NIFTY50', last...",2020-12-18 09:24:00.254020+00:00,"(0.36251267990655217, 0.999133868535508, 3174....","2,855.20","3,442.45","3,166.60",nan,"3,166.60",0.36
90,OPT,453924365,NIFTY50,20210107,"12,950.00",C,NIFTY2110712950CE,"Option(conId=453924365, symbol='NIFTY50', last...",2020-12-18 09:24:00.256017+00:00,"(0.1990428207190563, 0.912524938205372, 855.20...",735.15,971.20,884.30,nan,884.30,0.20
91,OPT,457243528,NIFTY50,20210107,"14,550.00",P,NIFTY2110714550PE,"Option(conId=457243528, symbol='NIFTY50', last...",2020-12-18 09:24:00.253023+00:00,"(0.14717971542767286, -0.9405081380732757, 780...",677.35,877.75,855.60,nan,855.60,0.15
92,OPT,454567506,NIFTY50,20210107,"14,150.00",P,NIFTY2110714150PE,"Option(conId=454567506, symbol='NIFTY50', last...",2020-12-18 09:24:00.252029+00:00,"(0.13859392953500743, -0.7878530744742307, 421...",276.30,521.85,552.00,nan,552.00,0.14


In [95]:
ct = df_opt_prices.contract.iloc[0]
with IB().connect(HOST, PORT, CID) as ib:
    p = ib.run(quick_price(ib, ct))

IndexError: list index out of range